<a href="https://colab.research.google.com/github/imusicmash/stanford_llm_python/blob/main/L1_Embeddings_api_intro_deeplearningdotai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started With Text Embeddings

#### Project environment setup

- Load credentials and relevant Python Libraries
- If you were running this notebook locally, you would first install Vertex AI.  In this classroom, this is already installed.
```Python
!pip install google-cloud-aiplatform
```


In [1]:
!pip install google-cloud-aiplatform

In [2]:
from utils import authenticate
credentials, PROJECT_ID = authenticate() # Get credentials and project ID

ModuleNotFoundError: No module named 'utils'

#### Enter project details

In [3]:
print(PROJECT_ID)

NameError: name 'PROJECT_ID' is not defined

In [ ]:
REGION = 'us-central1'

In [ ]:
# Import and initialize the Vertex AI Python SDK

import vertexai
vertexai.init(project = PROJECT_ID,
              location = REGION,
              credentials = credentials)

#### Use the embeddings model
- Import and load the model.

In [ ]:
from vertexai.language_models import TextEmbeddingModel

In [ ]:
embedding_model = TextEmbeddingModel.from_pretrained(
    "textembedding-gecko@001")

- Generate a word embedding

In [ ]:
embedding = embedding_model.get_embeddings(
    ["life"])

- The returned object is a list with a single `TextEmbedding` object.
- The `TextEmbedding.values` field stores the embeddings in a Python list.

In [ ]:
vector = embedding[0].values
print(f"Length = {len(vector)}")
print(vector[:10])

- Generate a sentence embedding.

In [ ]:
embedding = embedding_model.get_embeddings(
    ["What is the meaning of life?"])

In [ ]:
vector = embedding[0].values
print(f"Length = {len(vector)}")
print(vector[:10])

#### Similarity

- Calculate the similarity between two sentences as a number between 0 and 1.
- Try out your own sentences and check if the similarity calculations match your intuition.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
emb_1 = embedding_model.get_embeddings(
    ["What is the meaning of life?"]) # 42!

emb_2 = embedding_model.get_embeddings(
    ["How does one spend their time well on Earth?"])

emb_3 = embedding_model.get_embeddings(
    ["Would you like a salad?"])

vec_1 = [emb_1[0].values]
vec_2 = [emb_2[0].values]
vec_3 = [emb_3[0].values]

- Note: the reason we wrap the embeddings (a Python list) in another list is because the `cosine_similarity` function expects either a 2D numpy array or a list of lists.
```Python
vec_1 = [emb_1[0].values]
```

In [ ]:
print(cosine_similarity(vec_1,vec_2))
print(cosine_similarity(vec_2,vec_3))
print(cosine_similarity(vec_1,vec_3))

#### From word to sentence embeddings
- One possible way to calculate sentence embeddings from word embeddings is to take the average of the word embeddings.
- This ignores word order and context, so two sentences with different meanings, but the same set of words will end up with the same sentence embedding.

In [ ]:
in_1 = "The kids play in the park."
in_2 = "The play was for kids in the park."

- Remove stop words like ["the", "in", "for", "an", "is"] and punctuation.

In [ ]:
in_pp_1 = ["kids", "play", "park"]
in_pp_2 = ["play", "kids", "park"]

- Generate one embedding for each word.  So this is a list of three lists.

In [ ]:
embeddings_1 = [emb.values for emb in embedding_model.get_embeddings(in_pp_1)]

- Use numpy to convert this list of lists into a 2D array of 3 rows and 768 columns.

In [ ]:
import numpy as np
emb_array_1 = np.stack(embeddings_1)
print(emb_array_1.shape)

In [ ]:
embeddings_2 = [emb.values for emb in embedding_model.get_embeddings(in_pp_2)]
emb_array_2 = np.stack(embeddings_2)
print(emb_array_2.shape)

- Take the average embedding across the 3 word embeddings
- You'll get a single embedding of length 768.

In [ ]:
emb_1_mean = emb_array_1.mean(axis = 0)
print(emb_1_mean.shape)

In [ ]:
emb_2_mean = emb_array_2.mean(axis = 0)

- Check to see that taking an average of word embeddings results in two sentence embeddings that are identical.

In [ ]:
print(emb_1_mean[:4])
print(emb_2_mean[:4])

#### Get sentence embeddings from the model.
- These sentence embeddings account for word order and context.
- Verify that the sentence embeddings are not the same.

In [ ]:
print(in_1)
print(in_2)

In [ ]:
embedding_1 = embedding_model.get_embeddings([in_1])
embedding_2 = embedding_model.get_embeddings([in_2])

In [ ]:
vector_1 = embedding_1[0].values
print(vector_1[:4])
vector_2 = embedding_2[0].values
print(vector_2[:4])